## Лабораторная работа №6 Извлечение именованных сущностей (NER)
#### Предмет: Методы интеллектуального анализа текстов
#### Студент: Леонова Алина, СБ № 1032212306, НФИмд-01-21

### 6.3. Улучшить качество базовой предсказательной CRF-модели на тестовой выборке за счет добавления и модификации признаков

### 6.4.	Улучшить качество базовой предсказательной LSTM-CRF-модели на тестовой выборке за счет подбора предобученных эмбеддингов. Взять две эмбединг-модели и сравнить результаты.

Загрузка данных:

In [2]:
! wget https://www.dropbox.com/s/iuwsx5pmfhkk0w2/ner_dataset.csv

--2022-10-17 00:52:46--  https://www.dropbox.com/s/iuwsx5pmfhkk0w2/ner_dataset.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/iuwsx5pmfhkk0w2/ner_dataset.csv [following]
--2022-10-17 00:52:46--  https://www.dropbox.com/s/raw/iuwsx5pmfhkk0w2/ner_dataset.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc7f0c7c15c395170582ffae6a6.dl.dropboxusercontent.com/cd/0/inline/Bu8ptAghSa2PDCkf-yrZKmL-FBo6gsS3_y7QHTVgikPlrpJZnzqSf31_4jYTafCp8-ucmhXo4X-wHlvjO5kAI3lCgHOFLjnddVJseWMwucwu41X7osXhbzqHo4zwpZ6sSZDoLjfIA4LW5yFd_i2CcOWFXLfbmB9Khzu86VtLvOnZnA/file# [following]
--2022-10-17 00:52:46--  https://ucc7f0c7c15c395170582ffae6a6.dl.dropboxusercontent.com/cd/0/inline/Bu8ptAghSa2PDCkf-yrZKmL-FBo6gsS3_y7QHTVgikPlrpJZnzqSf31_4jYTafCp8-ucmhX

#### Baseline для 6.3

In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv("ner_dataset.csv", encoding="latin1")
data = data.fillna(method="ffill")

agg_func = lambda s: [[w, p, t] for w, p, t in zip(s["Word"].values.tolist(),
                                                   s["POS"].values.tolist(),
                                                   s["Tag"].values.tolist())]
grouped = data.groupby("Sentence #").apply(agg_func).values.tolist()

X_list = [[word[:2] for word in sentence] for sentence in grouped]
y_list = [[word[2] for word in sentence] for sentence in grouped]

In [4]:
data

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O


In [5]:
grouped[0]

[['Thousands', 'NNS', 'O'],
 ['of', 'IN', 'O'],
 ['demonstrators', 'NNS', 'O'],
 ['have', 'VBP', 'O'],
 ['marched', 'VBN', 'O'],
 ['through', 'IN', 'O'],
 ['London', 'NNP', 'B-geo'],
 ['to', 'TO', 'O'],
 ['protest', 'VB', 'O'],
 ['the', 'DT', 'O'],
 ['war', 'NN', 'O'],
 ['in', 'IN', 'O'],
 ['Iraq', 'NNP', 'B-geo'],
 ['and', 'CC', 'O'],
 ['demand', 'VB', 'O'],
 ['the', 'DT', 'O'],
 ['withdrawal', 'NN', 'O'],
 ['of', 'IN', 'O'],
 ['British', 'JJ', 'B-gpe'],
 ['troops', 'NNS', 'O'],
 ['from', 'IN', 'O'],
 ['that', 'DT', 'O'],
 ['country', 'NN', 'O'],
 ['.', '.', 'O']]

In [6]:
X_list[0], y_list[0]

([['Thousands', 'NNS'],
  ['of', 'IN'],
  ['demonstrators', 'NNS'],
  ['have', 'VBP'],
  ['marched', 'VBN'],
  ['through', 'IN'],
  ['London', 'NNP'],
  ['to', 'TO'],
  ['protest', 'VB'],
  ['the', 'DT'],
  ['war', 'NN'],
  ['in', 'IN'],
  ['Iraq', 'NNP'],
  ['and', 'CC'],
  ['demand', 'VB'],
  ['the', 'DT'],
  ['withdrawal', 'NN'],
  ['of', 'IN'],
  ['British', 'JJ'],
  ['troops', 'NNS'],
  ['from', 'IN'],
  ['that', 'DT'],
  ['country', 'NN'],
  ['.', '.']],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O'])

In [7]:
data.Tag.unique()

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
from sklearn.model_selection import train_test_split

data_train, data_test, y_train, y_test = train_test_split(X_list, y_list, test_size=0.2, random_state=1337)

In [9]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        ### Your code goes here ###
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            ### Your code goes here ###
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            ### Your code goes here ###
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [10]:
X_train = [sent2features(s) for s in data_train]
X_test = [sent2features(s) for s in data_test]

In [11]:
X_train[0][0]

{'word.lower()': 'the',
 'word.isupper()': False,
 'word.istitle()': True,
 'word.isdigit()': False,
 'postag': 'DT',
 'BOS': True,
 '+1:word.lower()': 'student',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False}

Преобразую слова и теги к численному виду для ускорения и улучшения работы модели

In [12]:
# преобразование слов и тегов к численному виду
from future.utils import iteritems
word2idx = {w: i+2 for i, w in enumerate(data.Word.unique())}
tag2idx = {t: i+1 for i, t in enumerate(data.Tag.unique())}
idx2tag = {v: k for k, v in iteritems(tag2idx)}

word2idx["--UNKNOWN_WORD--"]= 0
word2idx["--PADDING--"]= 1
tag2idx["--PADDING--"] = 0

idx2word = {idx: word for word, idx in word2idx.items()}


# проверка слова
test_word = "German"

test_word_idx = word2idx[test_word]
test_word_lookup = idx2word[test_word_idx]

print("The index of the word {} is {}.".format(test_word, test_word_idx))
print("The word with index {} is {}.".format(test_word_idx, test_word_lookup))


The index of the word German is 205.
The word with index 205 is German.


In [13]:
X = [[word[0] for word in sentence] for sentence in grouped]
y = [[word[2] for word in sentence] for sentence in grouped]
print("X[0]:", X[0])
print("y[0]:", y[0])

X[0]: ['Thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.']
y[0]: ['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [14]:
# будут использоваться дальше и в параметрах модели 
word_counts = data.groupby("Sentence #")["Word"].agg(["count"])
MAX_SENTENCE = word_counts.max()[0]
WORD_COUNT = len(word2idx)
TAG_COUNT = len(tag2idx)
DENSE_EMBEDDING = 50
LSTM_UNITS = 50
LSTM_DROPOUT = 0.1
DENSE_UNITS = 100
BATCH_SIZE = 256
MAX_EPOCHS = 5

In [15]:
# упрощение данных, представление цифрами
X = [[word2idx[word] for word in sentence] for sentence in X]
y = [[tag2idx[tag] for tag in sentence] for sentence in y]

# выравнивание длин предложений
X = [sentence + [word2idx["--PADDING--"]] * (MAX_SENTENCE - len(sentence)) for sentence in X]
y = [sentence + [tag2idx["--PADDING--"]] * (MAX_SENTENCE - len(sentence)) for sentence in y]

print("X[0]:", X[0])
print("y[0]:", y[0])
print("X[5555]:", X[5555])
print("y[5555]:", y[5555])

X[0]: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 11, 17, 3, 18, 19, 20, 21, 22, 23, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y[0]: [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
X[5555]: [10483, 6712, 11, 939, 3223, 1372, 9027, 6709, 13, 11, 936, 95, 298, 299, 1059, 3180, 1245, 299, 21, 3, 11, 3172, 3235, 11, 1930, 1665, 3, 47, 622, 193, 11, 19974, 11615, 1345, 19975, 23, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [16]:
y = [ np.eye(TAG_COUNT)[sentence] for sentence in y]
print("X[0]:", X[0], len(X), len(X[0]))
print("y[0]:", y[0], len(y), len(y[0]))

X[0]: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 11, 17, 3, 18, 19, 20, 21, 22, 23, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 47959 104
y[0]: [[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]] 47959 104


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train = [np.array(val) for val in X_train]
X_test = [np.array(val) for val in X_test]
y_train = [np.array(val) for val in y_train]
y_test = [np.array(val) for val in y_test]

print("Number of sentences in the training dataset: {}".format(len(X_train)))
print("Number of sentences in the test dataset : {}".format(len(X_test)))

print("X[0]:", X_train[0], len(X_train), len(X_train[2]))
print("y[0]:", y_train[0], len(y_train), len(y_train[2]))

Number of sentences in the training dataset: 43163
Number of sentences in the test dataset : 4796
X[0]: [ 7624   127  6442    21   129   182 23499  1904    65 16570  3292    23
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1] 43163 104
y[0]: [[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]] 43163 104


In [18]:
! pip install git+https://www.github.com/keras-team/keras-contrib.git
! pip install plot.keras.history

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-042ulwcm
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-042ulwcm
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import pickle
import operator
import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from plot_keras_history import plot_history
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from keras_contrib.utils import save_load_utils

from keras import layers
from keras import optimizers

from keras.models import Model
from keras import Input

from keras_contrib.layers import CRF
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib import losses
from keras_contrib import metrics

In [20]:
# Инициализировать и обучить модель, оценить её качество, прокомментировать, получается ли лучшее качество с вашими признаками или без них и почему.

In [21]:
# Инициализация модели
input_layer = layers.Input(shape=(MAX_SENTENCE,))

model = layers.Embedding(
    input_dim = WORD_COUNT, 
    output_dim = DENSE_EMBEDDING, 
    embeddings_initializer="uniform", 
    input_length=MAX_SENTENCE
    )(input_layer)

model = layers.Bidirectional(
    layers.LSTM(LSTM_UNITS, recurrent_dropout=LSTM_DROPOUT, return_sequences=True)
    )(model)

model = layers.TimeDistributed(
    layers.Dense(DENSE_UNITS, activation="relu")
    )(model)

crf_layer = CRF(units=TAG_COUNT)
output_layer = crf_layer(model)

ner_model = Model(input_layer, output_layer)

loss = losses.crf_loss
acc_metric = metrics.crf_accuracy
opt = optimizers.Adam(learning_rate=0.001)

ner_model.compile(optimizer=opt, loss=loss, metrics=[acc_metric])

ner_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 104)]             0         
                                                                 
 embedding (Embedding)       (None, 104, 50)           1759000   
                                                                 
 bidirectional (Bidirectiona  (None, 104, 100)         40400     
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 104, 100)         10100     
 ibuted)                                                         
                                                                 
 crf (CRF)                   (None, 104, 18)           2178      
                                                                 
Total params: 1,811,678
Trainable params: 1,811,678
Non-train

In [22]:
# Обучение модели
history = ner_model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=MAX_EPOCHS, validation_split=0.1, verbose=2)

Epoch 1/5


ValueError: ignored

Не удалось исправить и довести до работающего состояния даже на примере ner датасета (процесс заввершается с ошибкой после 15-17 минут ожидания), ниже смотрю на данные для лабораторной с наташей. 

In [ ]:
y_pred = ner_model.predict(X_test)

y_pred = np.argmax(y_pred, axis=2)

y_test = np.argmax(y_test, axis=2)

accuracy = (y_pred == y_test).mean()

print("Accuracy: {:.4f}/".format(accuracy))

In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred, average='macro')
print('\nf1_score = ', f1)

- В качестве решения любого задания <b>не принимается</b> модель с качеством менее 62.00% макроусредненной F1 на тесте. 
- <b>Можно</b> улучшать модели сверх предложенных условий: <b>добавлять свои признаки к указанным в задании</b>, изменять способ классификации и подбирать гиперпараметры. 
- Тестовые данные можно использовать только при оценке моделей.


Данные для обучения моделей:

In [23]:
! wget https://www.dropbox.com/s/t1gs701zvqaxqnk/rusentiment_random_posts.csv

--2022-10-17 01:13:30--  https://www.dropbox.com/s/t1gs701zvqaxqnk/rusentiment_random_posts.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/t1gs701zvqaxqnk/rusentiment_random_posts.csv [following]
--2022-10-17 01:13:31--  https://www.dropbox.com/s/raw/t1gs701zvqaxqnk/rusentiment_random_posts.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf2e302f60aca7740ba46207219.dl.dropboxusercontent.com/cd/0/inline/Bu_ilFLsatT7G1WaNRVWbNrqm0PLLWP0mwj3CF-tvWm3BEzINDpuN1NanMzW_yWSEIcqFCiPNQsozGBWtIjkeeYX_ZKiZgTaFw1yE9WXeD6Pz6zrXKx7owNx26vN6wJzgawuWzoWppyvaE9vgNRFxQ0m8i6CxYdq5JAXEei4iT45zg/file# [following]
--2022-10-17 01:13:31--  https://ucf2e302f60aca7740ba46207219.dl.dropboxusercontent.com/cd/0/inline/Bu_ilFLsatT7G1WaNRVWbNrqm0PLLWP0

Данные для тестирования моделей: 

In [24]:
! wget https://www.dropbox.com/s/gr4z1x39y1j6dtx/rusentiment_test.csv

--2022-10-17 01:14:03--  https://www.dropbox.com/s/gr4z1x39y1j6dtx/rusentiment_test.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/gr4z1x39y1j6dtx/rusentiment_test.csv [following]
--2022-10-17 01:14:03--  https://www.dropbox.com/s/raw/gr4z1x39y1j6dtx/rusentiment_test.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc723eaed800a03f8c67495f96fb.dl.dropboxusercontent.com/cd/0/inline/Bu_4DdtGsUsGQRahz3jWlL4wo7PhTKPTr8RBsHXsrMqywQjjWZw6NA-kd_KM6I6K0z9O7XYGCHJx7ByMhYvD6YrnCraFMD0FkPvOWhbqR1KeTEEqfM_ODAZHQG2XND-cLE9gMgKlWpAzk4EOPjNECrgIwXtY_MSQcqsM69Ks3bDSeA/file# [following]
--2022-10-17 01:14:04--  https://uc723eaed800a03f8c67495f96fb.dl.dropboxusercontent.com/cd/0/inline/Bu_4DdtGsUsGQRahz3jWlL4wo7PhTKPTr8RBsHXsrMqywQjjWZw6NA-k

In [25]:
import pandas as pd
import numpy as np

train = pd.read_csv("rusentiment_random_posts.csv")
train = train.fillna(method="ffill")

test = pd.read_csv("rusentiment_test.csv")
test = test.fillna(method="ffill")

In [26]:
train

,label,text
0,negative,"А попа подозревала давно,что ты с кавказа..пер..."
1,speech,З прошедшим Днем Ангела))))))))
2,skip,Два дня до отлёта с острова!!!!!!!
3,negative,"Блин, почему эта жизнь столь не справедлива (((("
4,skip,где еще встречать свой день рождения как не на...
...,...,...
21263,neutral,"Анастасия, у меня есть друг, с которым вы хоро..."
21264,neutral,Колька пошли гулять!!?
21265,neutral,Ура! Золото дают бесплатно!Напиши это в 4 комм...
21266,speech,"С Праздником, Ксюнь! \nЖенского счастья тебе!\..."


In [27]:
test

,label,text
0,neutral,"Александр, тебе к лицу эта пушка :)\n"
1,positive,"Скоро ты вернешься домой, грязный, не бритый н..."
2,neutral,помниш...))
3,skip,Мы просто с [id12210212|Елисеем] психанули )...
4,speech,"ой, прости забыла поздравить, с праздничьком, ..."
...,...,...
2962,neutral,Коллективно жмём F13! И наблюдаем за результатом!
2963,neutral,ану ка быстро цоя добавь!
2964,skip,Ты одна половинка - я вторая половинка...... А...
2965,positive,Люблю кису)))


In [28]:
! pip install natasha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 34.4 MB 106 kB/s 
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 49 kB 6.4 MB/s 
     |████████████████████████████████| 41 kB 120 kB/s 
     |████████████████████████████████| 8.2 MB 41.2 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=02db97525fead0f4384868c69c9ea41c71adbe7521aa99ffba70e0d72968f728
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=75cca58d922c7d12874d333dc979f67f94270dd910b288784bb11f39c283afdd
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built intervaltree docopt
  Attempting uninstall: intervaltree
    Fou

In [29]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()
embadings = NewsEmbedding()

morph_tagger = NewsMorphTagger(embadings)
ner_tagger = NewsNERTagger(embadings)
names_extractor = NamesExtractor(morph_vocab)

for sent in test.text[:20]:
  doc = Doc(sent)
  doc.segment(segmenter)
  doc.tag_ner(ner_tagger)
  doc.tag_morph(morph_tagger)
  doc.ner.print()

Александр, тебе к лицу эта пушка :)
PER──────                          
Скоро ты вернешься домой, грязный, не бритый но такой любимый ❤
помниш...))
Мы просто с  [id12210212|Елисеем]  психанули )))))
                         PER────                  
К новому году в бункере готовы!))))))
Да, мы любим молоко! Когда стареешь -  переходишь на него вместо 
огненной воды🍹
ой, прости забыла поздравить, с праздничьком, да ! ))))
Наши красавцы. 1:3 в первом периоде и 7:3 в итоге.
20% усилий приносят 80% результата
ты для меня самый родной!!!
нельзя общаться с людьми, которые нуждаются в тебе меньше чем ты в 
них... это самоунижение
мы с Мангаром решили наведаться в гости к тебе)
     PER─────                                  
Друзья, скиньте клёвой музычки, в тачке не чего слушать???
Селфи дня ))
Проект Анастасия. Часть 1 — на Яндекс.Видео
       PER──────               ORG─────────
ВОТ ОДИН ИЗ ИНТЕРЕСНЫХ КОММЕНТАРИЕВ К ФИЛЬМУ:
ORG                    ORG─────────          
к сожалению отсутстви

In [30]:
for sent in train.text[:25]:
  doc = Doc(sent)
  doc.segment(segmenter)
  doc.tag_ner(ner_tagger)
  doc.tag_morph(morph_tagger)
  doc.ner.print()

А попа подозревала давно,что ты с кавказа..перестану общаться с тобой
З прошедшим Днем Ангела))))))))
Два дня до отлёта с острова!!!!!!!
Блин, почему эта жизнь столь не справедлива ((((
где еще встречать свой день рождения как не на кладбище))))
Решите уравнение мужских дел для жизни :
Домов - 3
Деревьев - 5 
Сыновей - ?
патамушта я крутая баба! ееее!!
Просто пост :)
урря!я дождался этой овцыы)
С Днем Рождения желаю много счастья, любви и успехов во всем:) И еще с
 днем Защитника Отечества;))))))))
Все мы - украинцы , белорусы, россияне - мечтаем жить вместе в одной 
большой, богатой и очень дружной стране. В Канаде, например.
                                           LOC───           
Наша гордость !
Как мало надо для счастья. Как много надо для счастья...
Походу, первым кто поздравил меня с 1 апреля - это РЖД, 40 мин жду 
                                                   ORG             
электричку
познакомил свою девушку с Ксюшей
                          PER───
сестричка* Ксюшка)